In [1]:
import cv2
import numpy as np
import time
import imutils
from joblib import load
from dataset_io import *
import timeit
from tqdm import tqdm
import psutil
from datetime import datetime
import cProfile

In [2]:
def pyramid(image, split_channels, initial_scale, final_scale, scale=1.5):
    original_w=image.shape[1]
    curr_scale=initial_scale
    while curr_scale>final_scale/scale:
        w = int(original_w * curr_scale)
        yield imutils.resize(image, width=w), [imutils.resize(ch, width=w) for ch in split_channels],curr_scale
        curr_scale/=scale
        
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [3]:
def area(rect):
    ((x,y),(x2,y2))=rect
    return (x2-x)*(y2-y)

def calc_overlapping(a,b, fun):
    dx = min(a[1][0], b[1][0]) - max(a[0][0], b[0][0])
    dy = min(a[1][1], b[1][1]) - max(a[0][1], b[0][1])
    max_area=fun([area(a), area(b)])
    overlap_area=dx*dy
    return overlap_area/max_area if ((dx>=0) and (dy>=0)) else 0

def scale_many(vals, scale):
    return [int(val/scale) for val in vals]

In [4]:
(winW, winH) = (120,24)
DS_min_winH_m=41
DS_max_winH_m=287
initial_scale=winH/DS_min_winH_m
final_scale=winH/DS_max_winH_m

os.chdir(PROJECT_PATH)
clf = load('rand_forest_clf.joblib') 
clf.set_params(n_jobs=-1)

def show(img, x, y, winW, winH, positives=[]):
    clone = img.copy()
    cv2.rectangle(clone, (x, y), (x + winW, y + winH), (255, 255, 255), 2)
    for p in positives:
        x,y=p
        cv2.rectangle(clone, (x, y), (x + winW, y + winH), (255, 255, 255), 2)
    cv2.imshow("Window", clone)
    cv2.waitKey(1)

def image_predict(X,IDX,rects_count):
    global glob_RES
    if len(X)!=0 and rects_count>0:
        pred=np.array([p[1] for p in clf.predict_proba(X)]) # weź tylko te powyżej 0.9, w IDX też
        max_pred_ids=[]
        max_preds=[]
        max_pred_IDXs=[]
        for i in range(rects_count):
            print("pred/IDX", pred.shape, IDX.shape)
            max_pred_id=np.argmax(pred)
            max_pred=pred[max_pred_id]
            print("max_pred", max_pred)
            max_preds.append(max_pred)
            max_pred_IDX=IDX[max_pred_id]
            max_pred_IDXs.append(max_pred_IDX)
            
            [scene_name, filename, scale, (x,y)]=max_pred_IDX
            [xs,ys,ws,hs]=scale_many([x,y,winW,winH], scale)
            
            def not_within(row, x, y, h, w):
                a=((x,y),(x+w,y+h))
                
                [scene_name, filename, scale, (rx,ry)]=row
                [rx,ry,rw,rh]=scale_many([rx,ry,winW,winH], scale)
                b=((rx,ry),(rx+rw,ry+rh))
                
                return calc_overlapping(a,b,np.min) < 0.5

            not_within_bools = np.array([not_within(row, xs, ys, hs, ws) for row in IDX])
            pred=pred[not_within_bools]
            IDX=IDX[not_within_bools]
        
        max_preds=np.array([[e] for e in max_preds])
        res=np.append(max_pred_IDXs, max_preds, 1)
        
        glob_RES=np.vstack([glob_RES,res])
    
def predict(filename, scene_name, label_resolution, rects):
    X=[]
    IDX=np.empty((0,4), object)
    
    image=imread_resized(scene_name, filename, label_resolution)
    split_channels=read_split_channels(scene_name, filename)
    for resized_img, resized_split_ch, scale in pyramid(image, split_channels, initial_scale, final_scale, scale=1.15):
        resized_ch=np.dstack(resized_split_ch)
        for (x, y, window) in sliding_window(resized_img, stepSize=4, windowSize=(winW, winH)):
            # if the window does not meet our desired window size, ignore it
            if window.shape[0] != winH or window.shape[1] != winW:
                continue
            channels_window=resized_ch[y:y + winH, x:x + winW]
            features=channels_window.flatten()
            X.append(features)
            #IDX.append([scene_name, filename, scale, (x,y)])
            row=[scene_name, filename, scale, (x,y)]
            IDX=np.vstack([IDX, row])
    image_predict(X,IDX, len(rects))

In [5]:
glob_RES=np.empty((0,5), object)

glob_DATETIME = datetime.now().strftime("%d-%m-%Y_%H:%M:%S")

def predict_all():
    [predict(*row) for row in tqdm([row for row in walk_dataset() if row[1]=="warehouse_1"])]# and row[0]=="r_1_0.jpg"])]

In [ ]:
cProfile.runctx(
            'predict_all()',
            globals(),
            locals(),
            'myProfilingFile.pstats'
        )
os.chdir(PROJECT_PATH)
!python3 /usr/local/lib/python3.6/dist-packages/gprof2dot.py -f pstats myProfilingFile.pstats | dot -Tpng -o image_output.png

  0%|          | 0/114 [00:00<?, ?it/s]

pred/IDX (74066,) (74066, 4)
max_pred 1.0
pred/IDX (71171,) (71171, 4)
max_pred 0.9375


  1%|          | 1/114 [00:23<44:47, 23.79s/it]

pred/IDX (74066,) (74066, 4)
max_pred 1.0
pred/IDX (71283,) (71283, 4)
max_pred 0.9140625


  2%|▏         | 2/114 [00:47<44:17, 23.72s/it]

...

 23%|██▎       | 26/114 [09:20<33:13, 22.65s/it]

pred/IDX (74066,) (74066, 4)
max_pred 1.0
pred/IDX (71283,) (71283, 4)
max_pred 0.9375


 24%|██▎       | 27/114 [09:44<33:34, 23.16s/it]

pred/IDX (74066,) (74066, 4)
max_pred 0.9140625


 25%|██▍       | 28/114 [10:09<34:06, 23.80s/it]

In [ ]:
def calc_overlappings(rects, pred_rects):
    global glob_overlappings    
    overlappings=[np.max([calc_overlapping(pred_rect,rect, np.max) for pred_rect in pred_rects]) for rect in rects]
    glob_overlappings.extend(overlappings)

def draw_predicted_rectangles(filename, scene_name, label_resolution, rects):
    global glob_RES
    
    img=imread_resized(scene_name, filename, label_resolution)
    img_rows=glob_RES[np.where((glob_RES[:,0] == scene_name) * (glob_RES[:,1] == filename))]
    
    #n_rects=len(rects)
    #max_rows=file_res[np.argsort(file_res[:,4])[-n_rects:]]
    pred_rects=[]
    rects=[add_margin(correct_rect_ratio(rect)) for rect in rects]
    for max_row in img_rows:
        [scene_name, filename, scale, (x,y), pred]=max_row
        [x,y,winW_s,winH_s]=scale_many([x,y,winW,winH], scale)
        cv2.rectangle(img, (x,y), (x + winW_s, y + winH_s), (0, 255, 0), 2)
        pred_rects.append(((x,y),(x+winW_s, y+winH_s)))
    for rect in rects: 
        ((x,y),(x2,y2))=rect
        cv2.rectangle(img, (x,y), (x2,y2), (255, 0, 0), 2)
        
#     for row in file_res:
#         [scene_name, filename, scale, (x,y), pred]=row
#         [x,y,winW_s,winH_s]=scale_many([x,y,winW,winH], scale)
#         #print(winH_s)
#         if pred>0.9:
#             cv2.rectangle(img, (x,y), (x + winW_s, y + winH_s), (0, 255, 0), 2)
    save_image(img, scene_name, filename, "predicted_labels")
    calc_overlappings(rects, pred_rects)

glob_overlappings=[]
[draw_predicted_rectangles(*row) for row in tqdm(walk_dataset()) if row[1]=="warehouse_1"]# and row[0]=="r_1_0.jpg"]
print("Mean overlapping: ",np.mean(glob_overlappings))

In [ ]:
# window only - 1.36s
# features only - 1.47s
# classify per image - 2.82s
# classify per scale - 8.47s
# dsiplay, features only - 52.64s